In [5]:
import sys
import os

import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import numpy as np

from mike_config import TFrame, crypto_lst, time_frame

sys.path.append(r"D:\GitHub\tryStock\model\util")
import plot_utils
from plot_utils import one_hot, plot_curve

INPUT_LEN = 160

In [2]:
from torch.utils.data import TensorDataset
from sklearn.model_selection import train_test_split
import featuretools as ft

def get_data(symbol: str, time_frame: str):
    file_path = 'D:/GitHub/tryStock/data/' + symbol.replace("/", "") + '/' + time_frame + '/featured.csv'
    df = pd.read_csv(file_path)
    return df

data = get_data(crypto_lst[0], time_frame[0].name)

In [3]:
data1 = data.drop(data.columns[120:160], axis=1)
data1

,open0,high0,low0,close0,volume0,open1,high1,low1,close1,volume1,...,high28,low28,close28,volume28,open29,high29,low29,close29,volume29,prediction
0,9343.99,9345.00,9335.00,9342.00,39.174849,9342.00,9344.00,9336.08,9340.17,57.982222,...,9450.00,9414.48,9435.01,147.591771,9435.57,9446.07,9421.68,9433.72,111.563262,2
1,16919.70,16927.53,16915.76,16925.78,602.397270,16925.55,16939.00,16915.00,16932.84,995.653940,...,17008.40,16991.91,16993.73,1239.900520,16994.12,16998.66,16990.34,16993.61,620.108180,2
2,22823.50,22847.75,22819.79,22843.44,207.684440,22843.44,22875.00,22830.67,22835.89,485.292770,...,22780.00,22720.00,22760.03,250.154060,22760.03,22783.00,22735.51,22751.83,213.239800,1
3,63158.74,63326.80,63158.73,63221.87,225.971895,63221.86,63246.25,63066.06,63072.28,156.609650,...,63061.81,62851.09,63056.19,292.155324,63056.20,63211.70,63053.41,63203.59,414.004724,1
4,8846.08,8858.50,8846.08,8854.00,152.264200,8854.06,8860.68,8848.78,8859.62,134.643439,...,8844.92,8835.97,8838.63,109.258707,8838.63,8848.82,8838.41,8843.91,117.590723,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571423,3527.85,3528.90,3523.12,3525.21,92.071247,3525.10,3527.22,3523.05,3524.02,50.470263,...,3534.80,3530.00,3532.84,63.102864,3532.80,3535.00,3531.97,3534.27,29.698826,1
571424,9610.03,9621.73,9607.28,9621.40,86.042948,9620.88,9621.55,9610.25,9616.83,93.361273,...,9601.86,9594.62,9601.35,57.199037,9601.35,9633.11,9601.35,9628.19,324.035131,2
571425,35848.20,35886.88,35788.25,35822.33,425.228634,35822.33,35914.93,35724.72,35888.09,529.474273,...,36334.75,36223.90,36304.99,205.225930,36306.12,36366.15,36260.08,36285.08,201.508566,1
571426,5058.78,5072.16,5057.61,5070.37,92.217737,5068.73,5075.33,5068.73,5073.35,67.870803,...,5141.25,5125.46,5132.41,97.197409,5131.30,5137.91,5123.30,5136.99,160.715247,1


In [6]:
np_arr = np.arange(0, data1.shape[0])
data1.insert(0, "index", pd.DataFrame(np_arr))
data1

,index,open0,high0,low0,close0,volume0,open1,high1,low1,close1,...,high28,low28,close28,volume28,open29,high29,low29,close29,volume29,prediction
0,0,9343.99,9345.00,9335.00,9342.00,39.174849,9342.00,9344.00,9336.08,9340.17,...,9450.00,9414.48,9435.01,147.591771,9435.57,9446.07,9421.68,9433.72,111.563262,2
1,1,16919.70,16927.53,16915.76,16925.78,602.397270,16925.55,16939.00,16915.00,16932.84,...,17008.40,16991.91,16993.73,1239.900520,16994.12,16998.66,16990.34,16993.61,620.108180,2
2,2,22823.50,22847.75,22819.79,22843.44,207.684440,22843.44,22875.00,22830.67,22835.89,...,22780.00,22720.00,22760.03,250.154060,22760.03,22783.00,22735.51,22751.83,213.239800,1
3,3,63158.74,63326.80,63158.73,63221.87,225.971895,63221.86,63246.25,63066.06,63072.28,...,63061.81,62851.09,63056.19,292.155324,63056.20,63211.70,63053.41,63203.59,414.004724,1
4,4,8846.08,8858.50,8846.08,8854.00,152.264200,8854.06,8860.68,8848.78,8859.62,...,8844.92,8835.97,8838.63,109.258707,8838.63,8848.82,8838.41,8843.91,117.590723,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571423,571423,3527.85,3528.90,3523.12,3525.21,92.071247,3525.10,3527.22,3523.05,3524.02,...,3534.80,3530.00,3532.84,63.102864,3532.80,3535.00,3531.97,3534.27,29.698826,1
571424,571424,9610.03,9621.73,9607.28,9621.40,86.042948,9620.88,9621.55,9610.25,9616.83,...,9601.86,9594.62,9601.35,57.199037,9601.35,9633.11,9601.35,9628.19,324.035131,2
571425,571425,35848.20,35886.88,35788.25,35822.33,425.228634,35822.33,35914.93,35724.72,35888.09,...,36334.75,36223.90,36304.99,205.225930,36306.12,36366.15,36260.08,36285.08,201.508566,1
571426,571426,5058.78,5072.16,5057.61,5070.37,92.217737,5068.73,5075.33,5068.73,5073.35,...,5141.25,5125.46,5132.41,97.197409,5131.30,5137.91,5123.30,5136.99,160.715247,1


In [13]:
import featuretools as ft

es = ft.EntitySet(id="my_set")
es = es.add_dataframe(dataframe_name='my_df', dataframe=data1, index="index")

feature_matrix, feature_defs = ft.dfs(entityset=es, target_dataframe_name='my_df')
feature_matrix

C:\Users\mikeq\anaconda3\lib\site-packages\featuretools\entityset\entityset.py:754: UserWarning: A Woodwork-initialized DataFrame was provided, so the following parameters were ignored: index
  warnings.warn(
C:\Users\mikeq\anaconda3\lib\site-packages\featuretools\synthesis\deep_feature_synthesis.py:169: UserWarning: Only one dataframe in entityset, changing max_depth to 1 since deeper features cannot be created
  warnings.warn(


,open0,high0,low0,close0,volume0,open1,high1,low1,close1,volume1,...,high28,low28,close28,volume28,open29,high29,low29,close29,volume29,prediction
index,,,,,,,,,,,,,,,,,,,,,
0,9343.99,9345.00,9335.00,9342.00,39.174849,9342.00,9344.00,9336.08,9340.17,57.982222,...,9450.00,9414.48,9435.01,147.591771,9435.57,9446.07,9421.68,9433.72,111.563262,2
1,16919.70,16927.53,16915.76,16925.78,602.397270,16925.55,16939.00,16915.00,16932.84,995.653940,...,17008.40,16991.91,16993.73,1239.900520,16994.12,16998.66,16990.34,16993.61,620.108180,2
2,22823.50,22847.75,22819.79,22843.44,207.684440,22843.44,22875.00,22830.67,22835.89,485.292770,...,22780.00,22720.00,22760.03,250.154060,22760.03,22783.00,22735.51,22751.83,213.239800,1
3,63158.74,63326.80,63158.73,63221.87,225.971895,63221.86,63246.25,63066.06,63072.28,156.609650,...,63061.81,62851.09,63056.19,292.155324,63056.20,63211.70,63053.41,63203.59,414.004724,1
4,8846.08,8858.50,8846.08,8854.00,152.264200,8854.06,8860.68,8848.78,8859.62,134.643439,...,8844.92,8835.97,8838.63,109.258707,8838.63,8848.82,8838.41,8843.91,117.590723,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571423,3527.85,3528.90,3523.12,3525.21,92.071247,3525.10,3527.22,3523.05,3524.02,50.470263,...,3534.80,3530.00,3532.84,63.102864,3532.80,3535.00,3531.97,3534.27,29.698826,1
571424,9610.03,9621.73,9607.28,9621.40,86.042948,9620.88,9621.55,9610.25,9616.83,93.361273,...,9601.86,9594.62,9601.35,57.199037,9601.35,9633.11,9601.35,9628.19,324.035131,2
571425,35848.20,35886.88,35788.25,35822.33,425.228634,35822.33,35914.93,35724.72,35888.09,529.474273,...,36334.75,36223.90,36304.99,205.225930,36306.12,36366.15,36260.08,36285.08,201.508566,1


In [51]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(INPUT_LEN, 16)
        self.fc2 = nn.Linear(16, 2)
        self.fc3 = nn.Linear(2, 1)
        # self.fc4 = nn.Linear(32, 16)
        # self.fc5 = nn.Linear(16, 8)
        # self.fc6 = nn.Linear(8, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = x.float()
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # x = F.relu(self.fc3(x))
        # x = F.relu(self.fc4(x))
        # x = F.relu(self.fc5(x))
        x = self.fc3(x)
        out = self.sigmoid(x)
        return out

In [47]:
from torch.utils.data import TensorDataset
from sklearn.model_selection import train_test_split

def load_dataset(symbol: str, time_frame: str, target: int, batch_size=512):
    file_path = 'D:/GitHub/tryStock/data/' + symbol.replace("/", "") + '/' + time_frame + '/featured.csv'
    df = pd.read_csv(file_path)
    true_df = df[df.iloc[:, -1] == target]
    true_df.iloc[:, -1] = 1
    other_df = df[df.iloc[:, -1] != target]
    false_df = other_df.sample(true_df.shape[0])
    false_df.iloc[:, -1] = 0
    train_df, test_df = train_test_split(pd.concat([true_df, false_df]), test_size=0.2, random_state=42)
    train_inputs = torch.tensor(train_df.iloc[:, :INPUT_LEN].values)
    train_outputs = torch.tensor(train_df.iloc[:, -1].values).float()
    test_inputs = torch.tensor(test_df.iloc[:, :INPUT_LEN].values)
    test_outputs = torch.tensor(test_df.iloc[:, -1].values).float()

    train_loader = DataLoader(TensorDataset(train_inputs, train_outputs), batch_size=batch_size,  shuffle=True)
    test_loader = DataLoader(TensorDataset(test_inputs, test_outputs), batch_size=batch_size,  shuffle=True)
    return train_loader, test_loader


In [26]:
train_loader3, test_loader3 = load_dataset(crypto_lst[0], time_frame[0].name, 3, 512)

In [68]:
def split_data(symbol: str, time_frame: str, target: int):
    file_path = 'D:/GitHub/tryStock/data/' + symbol.replace("/", "") + '/' + time_frame + '/featured.csv'
    df = pd.read_csv(file_path)
    true_df = df[df.iloc[:, -1] == target]
    true_df.loc[:, "prediction"] = 1
    other_df = df[df.iloc[:, -1] != target]
    false_df = other_df.sample(true_df.shape[0])
    false_df.loc[:, "prediction"] = 0
    train_df, test_df = train_test_split(pd.concat([true_df, false_df]), test_size=0.2, random_state=42)
    train_df.sample(frac=1, random_state=42).reset_index(drop=True)
    test_df.sample(frac=1, random_state=42).reset_index(drop=True)

    return train_df.iloc[:, :INPUT_LEN], train_df.iloc[:, -1], test_df.iloc[:, :INPUT_LEN], test_df.iloc[:, -1]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def logistic_train(target: int):
    x_train, y_train, x_test, y_test = split_data(crypto_lst[0], time_frame[0].name, 3)
    # Initialize the logistic regression model
    model = LogisticRegression(max_iter=99999)

    # Train the model
    model.fit(x_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(x_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    print("Current Target:", target)
    print("Accuracy:", accuracy)

for i in range(4):
    logistic_train(i)

In [73]:
from sklearn.ensemble import RandomForestClassifier

def split_data2(symbol: str, time_frame: str):
    file_path = 'D:/GitHub/tryStock/data/' + symbol.replace("/", "") + '/' + time_frame + '/featured.csv'
    df = pd.read_csv(file_path)
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
    x_train = train_df.iloc[:, :INPUT_LEN]
    y_train = train_df.iloc[:, -1]
    x_test = test_df.iloc[:, :INPUT_LEN]
    y_test = test_df.iloc[:, -1]

    return x_train, y_train, x_test, y_test

x_train, y_train, x_test, y_test = split_data2(crypto_lst[0], time_frame[0].name)


def rf_train():
    model = RandomForestClassifier(n_estimators=500, class_weight='balanced_subsample', max_depth=10, min_samples_split=2, random_state=42)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)


In [74]:
rf_train()

Accuracy: 0.43749890625273435


In [36]:
import numpy as np
from sklearn.metrics import confusion_matrix

def test(net, test_loader):
    total_correct = [0, 0, 0, 0]
    total_count = [0, 0, 0, 0]
    true_labels = np.array([])
    predicted_labels = np.array([])
    for value, label in test_loader:
        value = value.view(value.size(0), INPUT_LEN)
        out = net(value)
        norm_out = normalize(out, p=2.0, dim=1)
        scaled_tensor = (norm_out - norm_out.min()) / (norm_out.max() - norm_out.min())
        pred = scaled_tensor.argmax(dim=1)
        temp = pred.eq(label) #.sum().float()

        print(confusion_matrix(label.numpy(), pred.numpy()))

        true_labels = np.append(true_labels, label.numpy())
        predicted_labels = np.append(predicted_labels, pred.numpy())


    confusion_mat = confusion_matrix(true_labels, predicted_labels)
    print(f"confusion matrix: \n{confusion_mat}")
test(net, test_loader)

NameError: name 'test_loader' is not defined